In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
import json
import subprocess
import os
import gzip
import pandas as pd
import tarfile
import numpy as np
import math

# add library code to path
sys.path.insert(0, r'src/')

In [4]:
cfg = json.load(open("../config/config.json"))

In [5]:
def mkdir(d):
    if not os.path.exists(d):
        os.mkdir(d)

In [6]:
mkdir('../data')

file_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE44639&format=file'

import requests

r = requests.get(file_url, stream=True)

with open("../data/raw/GSE44639_RAW.tar", 'wb') as f:
    for chunk in r.iter_content():
        f.write(chunk)

In [30]:
def read_file_type(health_types, cell_types, tmp='../data/temp/'):
    tar = tarfile.open("../data/raw/GSE44639_RAW.tar")

    mkdir(tmp)
    tar.extractall(tmp)
    
    df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)', 'healthy', 'subset'])
    
    for name in tar.getnames():
        subset = name.split('_')[-1].split('.')[0]
        health_type = name[4:].split('_')[1][0]
        
        if subset in cell_types and health_type in health_types:
            df1 = pd.read_csv(gzip.open(tmp + name,'rt'), sep='\t')
            df1['healthy'] = 'M' in name[4:]
            df1['subset'] = subset
            
            if len(df1.columns) != len(df.columns):
                continue
            
            df = pd.concat([df, df1])
            
    return df

df = read_file_type(['M', 'P'], ['rTreg', 'aTreg', 'Naive', 'Tcm', 'Tem', 'Ttm'])

In [35]:
import math

def absF(df, miRNA_name):
    # get only rows of the miRNA
    miRNAdf = df[df['miRNA'] == miRNA_name]
    miRNAdf['RPM (reads per million)'] = miRNAdf['RPM (reads per million)'].apply(lambda x: np.log2(x))
    N = len(miRNAdf)
    
    print(N)
    # calculate numerator
    meanY = miRNAdf['RPM (reads per million)'].astype(int).mean()
    groups = miRNAdf[['healthy', 'subset']].drop_duplicates()
    K = len(groups)
    
    print(K)
    
    numerator = 0
    denominator = 0

    for row in groups.iterrows():
#         print(row, row[1][1])
        health = row[1][0]
        cellType = row[1][1]
#         print((miRNAdf['healthy'] == health) and (miRNAdf['subset'] == cellType))
        groupDF = miRNAdf[(miRNAdf['healthy'] == health) & (miRNAdf['subset'] == cellType)]
        # mean Yi
        meanYi = groupDF['RPM (reads per million)'].astype(int).mean()
        numerator += np.abs(meanYi - meanY)*len(groupDF)/(K - 1)

        for sample in groupDF.iterrows():
            denominator += np.abs(sample[1][5] - meanYi)/(N - K)
            
            print(denominator)
            
    return numerator / denominator

absF(df, 'hsa-mir-21')

75
12
0.031746031746031744
0.031746031746031744
0.031746031746031744
0.07078462886725868
0.11160095539787093
0.12725914699640134
0.14291733859493175
0.16411514850254996
0.1881791845314669
0.21355214373803422
0.25542555316978627
0.2884661646928796
0.2985575737866153
0.3567563202417905
0.3869197190883756
0.41708311793496067
0.4255381223784725
0.43858846995206346
0.4804497353727153
0.5332223105737536
0.5361582232836132
0.5425074296328195
0.5647296518550418
0.60452156607105
0.6140453755948595
0.6172199787694627
0.6203945819440658
0.623569185118669
0.6267437882932722
0.6394422009916848
0.6672975581688827
0.6831705740418985
0.6897584708558484
0.7056314867288642
0.7083287089739803
0.7196665774547059
0.740074740720012
0.7604829039853181
0.7652328756520939
0.7699828473188696
0.7813207157995952
0.8147588608323646
0.8368597754259892
0.8748635649893615
0.9086623461570529
0.9088821238736342
0.9829312136003889
1.0008569788950645
1.0053921262873549
1.0146255257468073
1.0490170601383417
1.079505888306

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


3.8693031899684347

In [59]:
# prefix M = healthy, prefix P = unhealthy
# miRNA only appear on one type of chromosome
healthy_naiveDF['RPM (reads per million)'] = healthy_naiveDF['RPM (reads per million)'].astype(int)
mean_healthy = healthy_naiveDF.groupby('miRNA').mean()

In [ ]:
healthy_naiveDF['RPM (reads per million)'] = healthy_naiveDF['RPM (reads per million)'].astype(int)
mean_healthy = healthy_naiveDF.groupby('miRNA').mean()

In [9]:
df['log2RPM'] = df['RPM (reads per million)']

TypeError: cannot convert the series to <class 'float'>

In [39]:
healthy_naiveDF

NameError: name 'healthy_naiveDF' is not defined